# CausalAttention

In [2]:
import torch
import torch.nn as nn

class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout) # New
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # New

    def forward(self, x):
        b, num_tokens, d_in = x.shape # New batch dimension b
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2) # Changed transpose
        attn_scores.masked_fill_(  # New, _ ops are in-place
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)  # `:num_tokens` to account for cases where the number of tokens in the batch is smaller than the supported context_size
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights) # New

        context_vec = attn_weights @ values
        return context_vec

In [5]:
batch_size = 4
num_tokens = 10  
d_in = 16
d_out = 3
batch = torch.randn(batch_size, num_tokens, d_in)

torch.manual_seed(123)
context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)
context_vecs = ca(batch)
print("context_vecs.shape:", context_vecs.shape)

context_vecs.shape: torch.Size([4, 10, 3])


In [ ]:
Query, Key, Value (QKV) তৈরি করা:
প্রতিটি ইনপুট টোকেনের জন্য Query (Q), Key (K), এবং Value (V) তৈরি করা হয়।
Query (Q): বর্তমান টোকেন কী খুঁজছে।
Key (K): অন্য টোকেনগুলোর প্রাসঙ্গিকতা (relevance)।
Value (V): সেই প্রাসঙ্গিক টোকেন থেকে প্রাপ্ত তথ্য।
কেন করি?
কারণ attention মেকানিজমের মাধ্যমে মডেলকে শেখাতে চাই কোন টোকেনগুলো অন্য টোকেনের সাথে কতটা সম্পর্কিত।
Dimension বিভাজন করা (Multi-Head):
ইনপুট ডেটার dimension num_heads দিয়ে ভাগ করে প্রতিটি head-এ আলাদা ফিচার শেখার সুযোগ তৈরি করা হয়।

কেন করি?
একাধিক head ব্যবহার করলে ইনপুট ডেটার বিভিন্ন প্যাটার্ন বা সম্পর্ক বুঝতে মডেল আরও ভালো হয়। উদাহরণস্বরূপ, একটি head subject-object সম্পর্ক শেখে, আরেকটি head tense বা সময় বোঝে।

Attention Scores গণনা:
               T
Score=Query⋅Key 


এটি মূলত একটি ডট প্রোডাক্ট যা বলে দেয়, বর্তমান টোকেন (Query) অন্যান্য টোকেন (Key)-এর সাথে কতটা সম্পর্কযুক্ত।
কেন করি?
এটি নিশ্চিত করে যে কোন টোকেনগুলোর উপর বেশি ফোকাস করতে হবে। স্কোর যত বেশি, ফোকাস তত বেশি।

Scaled এবং Softmax প্রয়োগ করা:

Scaled করার জন্য 
Score/(route)dimensionকরা হয়।
​
Softmax দিয়ে স্কোরগুলোকে probability distribution-এ রূপান্তর করা হয়।
কেন করি?

Scaled করার মাধ্যমে অত্যধিক বড় বা ছোট স্কোর থেকে numerical instability রোধ হয়।
Softmax প্রয়োগের মাধ্যমে নিশ্চিত করা হয় যে টোকেনগুলোর মধ্যে প্রায়োরিটি তৈরি হয়।
Mask প্রয়োগ করা:

ভবিষ্যতের টোকেনগুলোর উপর ফোকাস রোধ করতে একটি mask ব্যবহার করা হয়।
Masked এলিমেন্টগুলোর স্কোর -inf দিয়ে পূরণ করা হয়, যাতে সেগুলোর softmax মান শূন্য হয়।
কেন করি?
কারণ অনেক অ্যাপ্লিকেশন (যেমন: ভাষা মডেল) ভবিষ্যতের টোকেন দেখে ফলাফল তৈরি করতে পারে না। এটি শুধুমাত্র বর্তমান এবং অতীত টোকেনের ওপর নির্ভর করে।

Weighted Sum (Attention Output):
Context
=
Attention Weights
⋅
Value
Context=Attention Weights⋅Value

Attention weights ব্যবহার করে Value ভেক্টরের weighted sum নেওয়া হয়।
কেন করি?
এটি প্রতিটি টোকেনের জন্য কনটেক্সট তৈরি করে যা বলে দেয়, সিকোয়েন্সের অন্য টোকেনগুলো কতটা গুরুত্বপূর্ণ।

Head Outputs Concatenate করা:
প্রতিটি head থেকে পাওয়া আউটপুট একত্রিত করে একটি ফাইনাল ভেক্টর তৈরি করা হয়।

কেন করি?
এটি নিশ্চিত করে যে মডেল ইনপুট ডেটার বিভিন্ন দৃষ্টিকোণ (perspective) থেকে সম্পর্ক বোঝে।

Final Linear Projection:
আউটপুট ভেক্টরকে একটি লিনিয়ার লেয়ার দিয়ে প্রজেক্ট করা হয় যাতে চূড়ান্ত আকার (dimension) পাওয়া যায়।

কেন করি?
মডেলের জন্য আউটপুট সঠিক আকারে (dimension) নিয়ে আসা প্রয়োজন যাতে এটি পরবর্তী ধাপে ব্যবহারযোগ্য হয়।

In [6]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

In [7]:
a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573], #A
[0.8993, 0.0390, 0.9268, 0.7388],
[0.7179, 0.7058, 0.9156, 0.4340]],
[[0.0772, 0.3565, 0.1479, 0.5331],
[0.4066, 0.2318, 0.4545, 0.9737],
[0.4606, 0.5159, 0.4220, 0.5786]]]])

In [8]:
print(a @ a.transpose(2, 3))

tensor([[[[1.3208, 1.1631, 1.2879],
          [1.1631, 2.2150, 1.8424],
          [1.2879, 1.8424, 2.0402]],

         [[0.4391, 0.7003, 0.5903],
          [0.7003, 1.3737, 1.0620],
          [0.5903, 1.0620, 0.9912]]]])


In [9]:
first_head = a[0, 0, :, :]
first_res = first_head @ first_head.T
print("First head:\n", first_res)
second_head = a[0, 1, :, :]
second_res = second_head @ second_head.T
print("\nSecond head:\n", second_res)

First head:
 tensor([[1.3208, 1.1631, 1.2879],
        [1.1631, 2.2150, 1.8424],
        [1.2879, 1.8424, 2.0402]])

Second head:
 tensor([[0.4391, 0.7003, 0.5903],
        [0.7003, 1.3737, 1.0620],
        [0.5903, 1.0620, 0.9912]])


In [10]:
torch.manual_seed(123)
batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[ 0.1937, -0.7206],
         [-0.0246, -0.6600],
         [-0.0181, -0.7055],
         [ 0.2677, -0.5520],
         [ 0.2688, -0.6845],
         [ 0.3168, -0.6202],
         [ 0.2888, -0.6215],
         [ 0.2699, -0.7250],
         [ 0.3662, -0.6551],
         [ 0.2967, -0.6294]],

        [[ 1.9564,  0.4715],
         [ 0.8358, -0.1561],
         [ 0.8107, -0.2891],
         [ 0.6754, -0.3448],
         [ 0.5698, -0.5668],
         [ 0.5682, -0.5072],
         [ 0.5189, -0.5563],
         [ 0.6367, -0.4840],
         [ 0.6981, -0.4700],
         [ 0.4923, -0.6446]],

        [[ 0.6105, -0.8314],
         [ 0.5328, -0.4183],
         [ 0.6882, -0.3075],
         [ 0.5568, -0.2954],
         [ 0.5763, -0.2767],
         [ 0.6404, -0.2021],
         [ 0.6499, -0.2396],
         [ 0.6597, -0.2663],
         [ 0.6579, -0.3026],
         [ 0.6101, -0.3756]],

        [[ 1.0248, -0.2336],
         [ 0.5855, -0.5365],
         [ 0.6044, -0.3609],
         [ 0.3708, -0.5311],
        